In [ ]:
# https://sbert.net/docs/sentence_transformer/training_overview.html

In [1]:
import pandas as pd
import torch
dataset_path="../dataset/Step4_Single_Trigger_IoT_Rules.csv"

In [2]:
df=pd.read_csv(dataset_path,skiprows=0)
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,...,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,...,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button)
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,...,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,...,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,...,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice)
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,...,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2643,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,...,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.turn_off_all_hue', 'name...",2,WeMo Light Switch,Philips Hue,Lighting,Lighting,['Ambient Luminance'],Other Trigger
2644,9959,False,"[{'name': 'RSS Feed', 'module_name': 'feed', '...","When Adafruit releases a new product, blink!",Ktup6MV2-when-adafruit-releases-a-new-product-...,Ktup6MV2,10.0,"When Adafruit releases a new product, blink!",[{'id': '/triggers/feed.new_feed_item_matches'...,False,...,"['feed', 'blink1']",Polling Applets usually run within 1 hour,[{'id': '/triggers/feed.new_feed_item_matches'...,2,RSS Feed,blink(1),News & information,Routers & computer accessories,['Alert User'],Other Trigger
2645,9958,False,"[{'name': 'Date & Time', 'module_name': 'date_...",Stop procrastination.,btpdY3TR-being-more-productive-and-goal-driven,btpdY3TR,10.0,Being more productive and goal driven,[{'id': '/triggers/da

In [3]:
df=df[["name","goal"]]
df

,name,goal
0,Turn on/off your lights with one tap on your p...,['Ambient Luminance']
1,Automatically turn your lights on at sunset,['Ambient Luminance']
2,Turn your lights on automatically as you arriv...,['Ambient Luminance']
3,Tell Alexa to start a Harmony activity,['Control Hub']
4,Blink your Hue lights when your Amazon Alexa t...,['Alert User']
...,...,...
2643,Long press Wemo Switch = All Hue lights off,['Ambient Luminance']
2644,"When Adafruit releases a new product, blink!",['Alert User']
2645,Being more productive and goal driven,['Other']
2646,White Light on Weekday Mornings,['Ambient Atmosphere']


In [4]:
df["goal"].iloc[33],type(df["goal"].iloc[31])

("['Ambient Atmosphere','Alert User']", str)

In [5]:
#perform some string manipulation
df["goal_clean"]=df["goal"].apply(lambda x:x.replace("[","").replace("]","").replace(",","").replace("'"," ").strip())

C:\Users\nicleonard\AppData\Local\Temp\ipykernel_16996\2646672195.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["goal_clean"]=df["goal"].apply(lambda x:x.replace("[","").replace("]","").replace(",","").replace("'"," ").strip())


In [6]:
df["goal_clean"].iloc[33],type(df["goal_clean"].iloc[31])

('Ambient Atmosphere  Alert User', str)

### Create a dataset in the form (anchor, positive) pairs with anchor the "name" attribute and positive the "goal_clean" 

In [7]:
from datasets import Dataset

anchors = []
positives = []
# Open a file, do preprocessing, filtering, cleaning, etc.
# and append to the lists

dataset = Dataset.from_dict({
    "anchor": df["name"],
    "positive": df["goal_clean"],
})

In [8]:
dataset

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 2648
})

### Setup the finetuning step with the simplest approach (anchor, positive)+MultipleNegativesRankingLoss
### Other combination (dataset format + loss) can be implemented and evaluated.
#### (anchor, positive) pairs is compatible with MultipleNegativesRankingLoss
https://sbert.net/docs/sentence_transformer/loss_overview.html

In [9]:
train_test_split = dataset.train_test_split(test_size=0.3, seed=42)

# This will give you a dictionary with the split datasets
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [12]:
train_dataset , test_dataset

(Dataset({
     features: ['anchor', 'positive'],
     num_rows: 1853
 }),
 Dataset({
     features: ['anchor', 'positive'],
     num_rows: 795
 }))

## Create the training setup 
steps taken from https://sbert.net/docs/sentence_transformer/training_overview.html

In [11]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
# 1. Load a model to finetune with 2. (Optional) model card data
model = SentenceTransformer(
    "all-mpnet-base-v2",
    #model_card_data=SentenceTransformerModelCardData(
    #    language="en",
    #    license="apache-2.0",
    #    model_name="MPNet base trained on AllNLI triplets",
    #)
)

In [13]:
print(model)
print("Max tokens Sequence Length:", model.max_seq_length)# (longer inputs will be truncated!)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Max tokens Sequence Length: 384


In [16]:
from sentence_transformers.losses import MultipleNegativesRankingLoss
# 4. Define a loss function
loss = MultipleNegativesRankingLoss(model)

In [17]:
# 5. (Optional) Specify training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/mpnet-base-all-nli-triplet-4ep",
    # Optional training parameters:
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=10,
    run_name="mpnet-base-all-nli-triplet",  # Will be used in W&B if `wandb` is installed
    optim="adamw_torch_fused"                  # use fused adamw optimizer
)

In [18]:
# 7. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss=loss,
    #evaluator=dev_evaluator,
)
trainer.train()

Step,Training Loss,Validation Loss
100,0.812400,0.545472
200,0.477900,0.476823
300,0.332100,0.464811
400,0.129900,0.470791


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=464, training_loss=0.6386170732178564, metrics={'train_runtime': 731.9068, 'train_samples_per_second': 10.127, 'train_steps_per_second': 0.634, 'total_flos': 0.0, 'train_loss': 0.6386170732178564, 'epoch': 3.9741379310344827})

advice: The training show overfitting

In [19]:
# 8. Save the trained model
model.save_pretrained(args.output_dir+"/final")

In [20]:
#save the model in the root folder (just for a simpler path)
trainer.save_model()

In [25]:
# 9. (Optional) Push it to the Hugging Face Hub
#model.push_to_hub("mpnet-base-all-nli-triplet-TAP")

##  Get the finetuned model back

In [21]:
args,args.output_dir

(SentenceTransformerTrainingArguments(output_dir='models/mpnet-base-all-nli-triplet-4ep', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, eval_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=4, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.1, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='models/mpnet-base-all-nli-triplet-4ep\\runs\\Jan25_17-32-18_NICLEONARDW', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=10, logging_nan_inf_f

In [22]:
fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)

In [23]:
fine_tuned_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)